# v5 of data

In [3]:
import pandas as pd
from IPython.display import HTML

In [4]:
df = pd.read_csv('v4crimedata.csv')

## Steps for cleaning the Overall Race column

- for null values, fill with 'Unknown'

In [6]:
df['Overall Race'].isnull().sum()

93

In [7]:
df['Overall Race'] = df['Overall Race'].fillna('UNKNOWN')

## Steps for cleaning Victim Age column
- calculate the average/median age for each race, then use these values to fill null victim age values
- chose to go with median since it is less impacted by outliers

In [9]:
df[df['Victim Age'].isnull()]

,Incident UID,Agency,Incident Date,CIBRS Unique Offense ID,CIBRS Offense Code,CIBRS Offense Description,Victim UID,Victim Category,Victim Age,Overall Race,City,Zip Code,Census Tract,CIBRS Status,Domestic Violence Incident,Beat,BCS Area
5,941769,SHERIFF,6/5/21 9:20,941769-13B-973739,13B,Simple Assault,973739,L,NaN,HISPANIC,EL CAJON,92020.0,NaN,ProcessingComplete - Valid,False,24,DETENTION FACILITY
6,772344,SHERIFF,6/22/21 9:00,772344-13B-796205,13B,Simple Assault,796205,I,NaN,OTHER,OCEANSIDE,NaN,NaN,ProcessingComplete - Valid,False,29,DETENTION FACILITY
16,1438067,SHERIFF,10/12/22 9:00,1438067-13A-1306213,13A,Aggravated Assault,1306213,I,NaN,WHITE,OCEANSIDE,NaN,NaN,ProcessingComplete - Valid,False,29,DETENTION FACILITY
24,2325795,SHERIFF,6/22/24 15:24,2325795-13B-2040146,13B,Simple Assault,2040146,L,NaN,UNKNOWN,SANTEE,92071.0,NaN,ProcessingComplete - Valid,False,24,DETENTION FACILITY
79,440058,SAN DIEGO,1/24/21 23:40,440058-13B-452032,13B,Simple Assault,452032,I,NaN,HISPANIC,SAN DIEGO,92114.0,30.03,ProcessingComplete - Valid,True,433,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62221,2335000,SHERIFF,9/28/24 8:23,2335000-13B-2047871,13B,Simple Assault,2047871,I,NaN,UNKNOWN,SPRING VALLEY,91977.0,137.01,ProcessingComplete - Valid,True,614,RANCHO SAN DIEGO UNINC
62243,2336588,SAN DIEGO,9/29/24 2:25,2336588-13B-2049226,13B,Simple Assault,2049226,L,NaN,ASIAN,SAN DIEGO,92109.0,79.10,ProcessingComplete - Valid,False,122,NaN
62247,2349146,CHULA VISTA,9/29/24 9:22,2349146-13A-2059191,13A,Aggravated Assault,2059191,L,NaN,WHITE,CHULA VISTA,91910.0,124.01,ProcessingComplete - Valid,False,11,NaN
62249,2332842,SAN DIEGO,9/28/24 22:00,2332842-13B-2046082,13B,Simple Assault,2046082,I,NaN,BLACK,SAN DIEGO,92102.0,33.04,ProcessingComplete - Valid,True,446,NaN


In [10]:
df['Overall Race'].value_counts()

Overall Race
HISPANIC            26499
WHITE               21717
BLACK                8372
ASIAN                2977
OTHER                1327
MIDDLE EASTERN        915
PACIFIC ISLANDER      339
UNKNOWN                93
AMERICAN ASIAN         90
Name: count, dtype: int64

In [11]:
for i in df['Overall Race'].unique():
    race_mean = df[df['Overall Race'] == i]['Victim Age'].mean()
    race_median = df[df['Overall Race'] == i]['Victim Age'].median()
    print(f'{i}: mean {race_mean}, median {race_median}')

HISPANIC: mean 31.255380255801487, median 30.0
WHITE: mean 38.51660777385159, median 37.0
OTHER: mean 31.538594969644407, median 30.0
BLACK: mean 32.65304400241109, median 30.0
ASIAN: mean 36.7677286742035, median 35.0
UNKNOWN: mean 29.840579710144926, median 30.0
MIDDLE EASTERN: mean 33.2421875, median 33.0
PACIFIC ISLANDER: mean 31.544642857142858, median 30.0
AMERICAN ASIAN: mean 35.2247191011236, median 33.0


In [12]:
median_age_per_race = df.groupby('Overall Race')['Victim Age'].transform('median')

In [13]:
# Replace NaN values in 'Victim Age' with the median age of the corresponding race
df['Victim Age'] = df['Victim Age'].fillna(median_age_per_race)

In [14]:
# for comparison: filling null values only shifted means by around 0.01 each, but will have an impact on count

for i in df['Overall Race'].unique():
    race_mean = df[df['Overall Race'] == i]['Victim Age'].mean()
    race_median = df[df['Overall Race'] == i]['Victim Age'].median()
    print(f'{i}: mean {race_mean}, median {race_median}')

HISPANIC: mean 31.237141024189594, median 30.0
WHITE: mean 38.48224892941014, median 37.0
OTHER: mean 31.336850037678975, median 30.0
BLACK: mean 32.6286430960344, median 30.0
ASIAN: mean 36.73328854551562, median 35.0
UNKNOWN: mean 29.881720430107528, median 30.0
MIDDLE EASTERN: mean 33.237158469945356, median 33.0
PACIFIC ISLANDER: mean 31.530973451327434, median 30.0
AMERICAN ASIAN: mean 35.2, median 33.0


## Steps for cleaning Census Tract data

- are we going to use this column?

In [16]:
df['Census Tract'].isnull().sum()

2563

## Clean Zip Code column

In [18]:
df[df['Zip Code'].isnull()]['City'].value_counts()

City
CARLSBAD          30
LA MESA           21
OCEANSIDE         20
SANTEE            18
NATIONAL CITY      8
SAN DIEGO          8
VISTA              6
EL CAJON           5
CHULA VISTA        4
ESCONDIDO          3
SAN MARCOS         3
SPRING VALLEY      2
IMPERIAL BEACH     2
ALPINE             1
CAMP PENDLETON     1
ENCINITAS          1
Name: count, dtype: int64

In [19]:
zipcode_from_city = {
    'CARLSBAD': 92008,
    'LA MESA': 91942,
    'OCEANSIDE': 92054,
    'SANTEE': 92071,
    'NATIONAL CITY': 91950,
    'SAN DIEGO': 92101,
    'VISTA': 92081,
    'EL CAJON': 92020,
    'CHULA VISTA': 91910,
    'ESCONDIDO': 92025,
    'SAN MARCOS': 92069,
    'IMPERIAL BEACH': 91932,
    'SPRING VALLEY': 91977,
    'ALPINE': 91901,
    'CAMP PENDLETON': 92055,
    'ENCINITAS': 92024
}

In [20]:
# Fill in missing zip codes based on the city values
def fill_missing_zip(row):
    if pd.isnull(row['Zip Code']):
        return zipcode_from_city.get(row['City'], row['Zip Code'])
    return row['Zip Code']

df['Zip Code'] = df.apply(fill_missing_zip, axis=1)

In [21]:
df['Zip Code'].isnull().sum()

41

## Adding Health and Human Service Agency (HHSA) Region Column

In [23]:
central_zipcodes = [
    92101, 92102, 92103, 92104, 92105, 92113, 
    92114, 92115, 92116, 92134, 92139, 
    92182, 92108, 92120, 92119.0, 92140.0
]

In [24]:
east_zipcodes = [
    91901, 91905, 91906, 91916, 91917, 91931, 
    91934, 91935, 91941, 91942, 91945, 91948, 
    91962, 91963, 91977, 91978, 91980, 92019, 
    92020, 92021, 92040, 92071
]

In [25]:
north_central_zipcodes = [
    92037, 92093, 92106, 92107, 92109, 92110, 
    92111, 92117, 92121, 92122, 92123, 92124, 
    92126, 92130, 92131, 92145, 92161
]

In [26]:
north_coastal_zipcodes = [
    92007, 92008, 92009, 92010, 92011, 92014, 
    92024, 92054, 92055, 92056, 92057, 92058, 
    92067, 92075, 92081, 92083, 92091, 
    92672
]

In [27]:
north_inland_zipcodes = [
    92004, 92025, 92026, 92027, 
    92029, 92036, 92059, 92060, 92061, 92064, 
    92065, 92066, 92069, 92070, 92078, 92082, 
    92086, 92096, 92127, 92128, 92129, 92259, 
    92536, 92028, 92003, 92084
]

In [28]:
south_zipcodes = [
    91902, 91910, 91911, 91913, 91914, 91915, 
    91932, 91950, 92118, 92135, 92154, 92155, 
    92173, 92136
]

In [29]:
hhsa_regions = {
    'CENTRAL': central_zipcodes,
    'NORTH CENTRAL': north_central_zipcodes,
    'EAST': east_zipcodes,
    'NORTH COASTAL': north_coastal_zipcodes,
    'NORTH INLAND': north_inland_zipcodes,
    'SOUTH': south_zipcodes
}


In [30]:
def map_zip_to_region(zip_code):
    for region, zips in hhsa_regions.items():
        if zip_code in zips:
            return region

df['HHSA Region'] = df['Zip Code'].apply(map_zip_to_region)

In [31]:
df['HHSA Region'].isnull().sum()

41

In [32]:
# now the only missing values of hhsa are values where we're missing zip codes

df['Zip Code'].isnull().sum()
df

,Incident UID,Agency,Incident Date,CIBRS Unique Offense ID,CIBRS Offense Code,CIBRS Offense Description,Victim UID,Victim Category,Victim Age,Overall Race,City,Zip Code,Census Tract,CIBRS Status,Domestic Violence Incident,Beat,BCS Area,HHSA Region
0,661987,SAN DIEGO,2/18/21 17:40,661987-13A-679864,13A,Aggravated Assault,679864,I,22.0,HISPANIC,SAN DIEGO,92037.0,NaN,ProcessingComplete - Valid,True,115,NaN,NORTH CENTRAL
1,793674,SAN DIEGO,3/16/21 5:30,793674-11A-818279,11A,Forcible Rape,818279,I,41.0,WHITE,SAN DIEGO,92110.0,NaN,ProcessingComplete - Valid,False,625,NaN,NORTH CENTRAL
2,859375,NATIONAL CITY,3/22/21 14:25,859375-11A-887268,11A,Forcible Rape,887268,I,22.0,HISPANIC,NATIONAL CITY,91950.0,NaN,ProcessingComplete - Valid,False,999,NaN,SOUTH
3,658884,SAN DIEGO,3/24/21 12:00,658884-13B-676587,13B,Simple Assault,676587,I,45.0,HISPANIC,SAN DIEGO,92101.0,NaN,ProcessingComplete - Valid,True,523,NaN,CENTRAL
4,721771,CARLSBAD,5/4/21 18:54,721771-13B-743217,13B,Simple Assault,743217,I,28.0,HISPANIC,CARLSBAD,92011.0,NaN,ProcessingComplete - Valid,False,6,NaN,NORTH COASTAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62324,2332374,OCEANSIDE,9/28/24 19:20,2332374-13B-2045587,13B,Simple Assault,2045587,I,36.0,HISPANIC,OCEANSIDE,92054.0,184.00,ProcessingComplete - Valid,True,6,NaN,NORTH COASTAL
62325,2340093,SHERIFF,9/30/24 19:02,2340093-13B-2051941,13B,Simple Assault,2051941,I,14.0,WHITE,ENCINITAS,92024.0,171.07,ProcessingComplete - Valid,False,223,CITY OF ENCINITAS,NORTH COASTAL
62326,2345854,EL CAJON,9/27/24 14:15,2345854-13B-2056500,13B,Simple Assault,2056500,I,8.0,MIDDLE EASTERN,EL CAJON,92021.0,168.07,ProcessingComplete - Valid,False,3,NaN,EAST
62327,2332703,SAN DIEGO,9/30/24 0:56,2332703-13B-2045943,13B,Simple Assault,2045943,I,59.0,HISPANIC,SAN DIEGO,92113.0,48.00,ProcessingComplete - Valid,True,512,NaN,CENTRAL


In [33]:
# fixing typos in the original data where zip code != city

df.loc[(df['City'] == 'CAMP PENDLETON') & (df['Zip Code'] != 92055), 'Zip Code'] = 92055
df.loc[(df['City'] == 'RAMONA') & (df['Zip Code'] != 92065), 'Zip Code'] = 92065
df.loc[(df['Zip Code'] == 91942) & (df['HHSA Region'] == 'EAST'), 'City'] = 'LA MESA'
df.loc[(df['Zip Code'] == 92071) & (df['HHSA Region'] == 'EAST'), 'City'] = 'SANTEE'
df.loc[(df['Zip Code'] == 92115) & (df['HHSA Region'] == 'CENTRAL'), 'City'] = 'SAN DIEGO'
df.loc[(df['Zip Code'] == 92114) & (df['HHSA Region'] == 'CENTRAL'), 'City'] = 'SAN DIEGO'

In [34]:
df['HHSA Region'] = df['Zip Code'].apply(map_zip_to_region)

## Convert Incident Date to DateTime Data type

In [36]:
df['Incident Date'].unique()

array(['2/18/21 17:40', '3/16/21 5:30', '3/22/21 14:25', ...,
       '9/28/24 19:20', '9/30/24 0:56', '9/29/24 19:00'], dtype=object)

In [37]:
df['Incident Date'] = pd.to_datetime(df['Incident Date'], format='%m/%d/%y %H:%M')

In [38]:
df['Incident Date'].unique()

<DatetimeArray>
['2021-02-18 17:40:00', '2021-03-16 05:30:00', '2021-03-22 14:25:00',
 '2021-03-24 12:00:00', '2021-05-04 18:54:00', '2021-06-05 09:20:00',
 '2021-06-22 09:00:00', '2021-07-03 02:00:00', '2021-08-13 21:00:00',
 '2021-12-17 18:00:00',
 ...
 '2024-09-30 03:57:00', '2024-09-28 20:50:00', '2024-09-27 15:05:00',
 '2024-09-29 03:08:00', '2024-09-29 12:51:00', '2024-09-30 16:45:00',
 '2024-09-27 19:30:00', '2024-09-28 19:20:00', '2024-09-30 00:56:00',
 '2024-09-29 19:00:00']
Length: 52590, dtype: datetime64[ns]

## Export final v5 dataframe

In [40]:
# v5crimedata

df.to_csv('v5crimedata.csv', index=False)

In [41]:
# create download link for v5 of data

def create_download_link(filename):
    return HTML(f'<a href="{filename}" download>Download {filename}</a>')

create_download_link('v5crimedata.csv')

# v6 of data

In [43]:
v5 = df

In [44]:
v5['HHSA Region'].value_counts()

HHSA Region
CENTRAL          15470
EAST             11181
NORTH INLAND      9950
SOUTH             9588
NORTH COASTAL     8438
NORTH CENTRAL     7661
Name: count, dtype: int64

In [45]:
for i in v5['HHSA Region'].unique():
    print(f"HHSA region {i} contains cities {v5[v5['HHSA Region'] == i]['City'].value_counts()}")
    print()

HHSA region NORTH CENTRAL contains cities City
SAN DIEGO    7658
LA JOLLA        3
Name: count, dtype: int64

HHSA region SOUTH contains cities City
CHULA VISTA       4986
NATIONAL CITY     1830
SOUTH BAY         1782
IMPERIAL BEACH     673
BONITA             152
SAN DIEGO          110
LINCOLN ACRES       34
SAN YSIDRO          15
CORONADO             5
OTAY                 1
Name: count, dtype: int64

HHSA region CENTRAL contains cities City
SAN DIEGO    15470
Name: count, dtype: int64

HHSA region NORTH COASTAL contains cities City
OCEANSIDE             4323
VISTA                 1534
CARLSBAD              1510
ENCINITAS              702
SOLANA BEACH           113
DEL MAR                 79
SAN DIEGO               62
RANCHO SANTA FE         60
CAMP PENDLETON          28
SAN MARCOS              15
CARDIFF BY THE SEA       7
SAN CLEMENTE             5
Name: count, dtype: int64

HHSA region EAST contains cities City
EL CAJON         3727
SPRING VALLEY    1789
LA MESA          1669
SANTE

In [46]:
v5[(v5['City'] == 'SAN YSABEL')]

,Incident UID,Agency,Incident Date,CIBRS Unique Offense ID,CIBRS Offense Code,CIBRS Offense Description,Victim UID,Victim Category,Victim Age,Overall Race,City,Zip Code,Census Tract,CIBRS Status,Domestic Violence Incident,Beat,BCS Area,HHSA Region
17422,933111,SHERIFF,2021-12-31 17:00:00,933111-13B-964614,13B,Simple Assault,964614,I,28.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,True,826,RANCHITA,NORTH INLAND
18031,960092,SHERIFF,2022-02-01 19:27:00,960092-13C-993164,13C,Intimidation,993164,I,39.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,825,RANCHITA,NORTH INLAND
24308,1215055,SHERIFF,2022-06-01 20:24:00,1215055-13B-1137149,13B,Simple Assault,1137149,I,39.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,825,RANCHITA,NORTH INLAND
25592,1210121,SHERIFF,2022-06-09 22:53:00,1210121-13C-1132818,13C,Intimidation,1132818,I,57.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,826,RANCHITA,NORTH INLAND
26732,1285948,SHERIFF,2022-08-03 00:36:00,1285948-13B-1197494,13B,Simple Assault,1197494,I,40.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,True,826,RANCHITA,NORTH INLAND
37229,1590640,SHERIFF,2023-02-26 21:30:00,1590640-13B-1436881,13B,Simple Assault,1436881,I,49.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,826,RANCHITA,NORTH INLAND
40096,1738850,SHERIFF,2023-06-15 18:15:00,1738850-13B-1558167,13B,Simple Assault,1558167,I,7.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,826,RANCHITA,NORTH INLAND
40616,1738850,SHERIFF,2023-06-15 18:15:00,1738850-13B-1558169,13B,Simple Assault,1558169,I,16.0,ASIAN,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,826,RANCHITA,NORTH INLAND
43594,1814562,SHERIFF,2023-08-29 09:00:00,1814562-13B-1619293,13B,Simple Assault,1619293,I,49.0,HISPANIC,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,True,827,RANCHITA,NORTH INLAND
45038,1803097,SHERIFF,2023-08-10 20:15:00,1803097-13B-1610145,13B,Simple Assault,1610145,I,49.0,WHITE,SAN YSABEL,92070.0,209.03,ProcessingComplete - Valid,False,826,RANCHITA,NORTH INLAND


In [47]:
# remove typos in original data: San Ysabel should be Santa Ysabel

v5['City'] = v5['City'].replace('SAN YSABEL', 'SANTA YSABEL')

In [48]:
v6 = v5
v6.head()

,Incident UID,Agency,Incident Date,CIBRS Unique Offense ID,CIBRS Offense Code,CIBRS Offense Description,Victim UID,Victim Category,Victim Age,Overall Race,City,Zip Code,Census Tract,CIBRS Status,Domestic Violence Incident,Beat,BCS Area,HHSA Region
0,661987,SAN DIEGO,2021-02-18 17:40:00,661987-13A-679864,13A,Aggravated Assault,679864,I,22.0,HISPANIC,SAN DIEGO,92037.0,NaN,ProcessingComplete - Valid,True,115,NaN,NORTH CENTRAL
1,793674,SAN DIEGO,2021-03-16 05:30:00,793674-11A-818279,11A,Forcible Rape,818279,I,41.0,WHITE,SAN DIEGO,92110.0,NaN,ProcessingComplete - Valid,False,625,NaN,NORTH CENTRAL
2,859375,NATIONAL CITY,2021-03-22 14:25:00,859375-11A-887268,11A,Forcible Rape,887268,I,22.0,HISPANIC,NATIONAL CITY,91950.0,NaN,ProcessingComplete - Valid,False,999,NaN,SOUTH
3,658884,SAN DIEGO,2021-03-24 12:00:00,658884-13B-676587,13B,Simple Assault,676587,I,45.0,HISPANIC,SAN DIEGO,92101.0,NaN,ProcessingComplete - Valid,True,523,NaN,CENTRAL
4,721771,CARLSBAD,2021-05-04 18:54:00,721771-13B-743217,13B,Simple Assault,743217,I,28.0,HISPANIC,CARLSBAD,92011.0,NaN,ProcessingComplete - Valid,False,6,NaN,NORTH COASTAL


In [49]:
# v6crimedata

v6.to_csv('v6crimedata.csv', index=False)

In [50]:
create_download_link('v6crimedata.csv')

In [99]:
v7 = v6[v6['Victim Age'] > 18]

In [101]:
v7['City'].isnull().sum()

25

In [111]:
v7 = v7.dropna(subset=['City'])

In [113]:
v7.shape

(53803, 18)

In [115]:
# v7crimedata

v7.to_csv('v7crimedata.csv', index=False)

In [117]:
create_download_link('v7crimedata.csv')